In [1]:
suppressPackageStartupMessages({library(tidyverse)
library(ribge)})

In [ ]:
anos <- 2016:2017
pib <- replicate(length(anos), list())
for(i in 1:length(pib)){
  pib[[i]] <- pib_municipios(anos[i])
}
pib1 <- rbind(pib[[1]],pib[[2]])
files <- c("PIB dos Munic¡pios - base de dados 2010-2017.xls",
           "base_de_dados_2010_2017_xls.zip",
           "base_1999_2012_xlsx.zip",
           "IA59_BR_PERC.csv",
           "IA55_BR_PERC.csv",
           "SCN54_BR_PERC.csv",
           "PIB dos Munic¡pios 1999-2012 Base Dados.xlsx")
for(i in files){
  file.remove(i)
}

In [ ]:
#PIB estadual
pib_estado <- pib1 %>%
  group_by(ano,codigo_uf, nome_uf) %>%
  summarise(codigo_regiao = mean(codigo_regiao),
            pib_total = sum(pib_total),
            vab_agropecuaria = sum(vab_agropecuaria)/pib_total,
            vab_industria = sum(vab_industria)/pib_total,
            vab_servicos_exclusivo = sum(vab_servicos_exclusivo)/pib_total,
            vab_adm_publica = sum(vab_adm_publica)/pib_total,
            vab_total = sum(vab_total)/pib_total,
            impostos = sum(impostos)/pib_total,
            pib_per_capita = mean(pib_per_capita),
            impostos_per_capita = pib_per_capita/pib_total*impostos)

In [ ]:
#PIB Regional
pib_regiao <- pib_estado %>%
  group_by(ano,codigo_regiao) %>%
  summarise(pib_total = sum(pib_total),
            vab_agropecuaria = mean(vab_agropecuaria),
            vab_industria = mean(vab_industria),
            vab_servicos_exclusivo = mean(vab_servicos_exclusivo),
            vab_adm_publica = mean(vab_adm_publica),
            vab_total = mean(vab_total),
            impostos = mean(impostos),
            pib_per_capita = mean(pib_per_capita),
            impostos_per_capita = pib_per_capita/pib_total*impostos)

In [ ]:
pib_regiao_a <- pib_regiao %>%
  gather(key = "key", value = "value", 
         -c(ano,pib_total,pib_per_capita,codigo_regiao,vab_total))
for(i in 1:nrow(pib_regiao_a)){
  if(pib_regiao_a$codigo_regiao[i] == 1){
    pib_regiao_a$codigo_regiao[i] <- "Norte"
  }else if(pib_regiao_a$codigo_regiao[i] == 2){
    pib_regiao_a$codigo_regiao[i] <- "Nordeste"
  }else if(pib_regiao_a$codigo_regiao[i] == 3){
    pib_regiao_a$codigo_regiao[i] <- "Sudeste"
  }else if(pib_regiao_a$codigo_regiao[i] == 4){
    pib_regiao_a$codigo_regiao[i] <- "Sul"
  }else if(pib_regiao_a$codigo_regiao[i] == 5){
    pib_regiao_a$codigo_regiao[i] <- "Centro-Oeste"
  }
}

In [ ]:
ggplot(pib_regiao_a, aes(x = ano, y = value, fill = key))+
  geom_col(position = "dodge")+
  facet_wrap(~codigo_regiao, ncol = 2, scales = "free")